# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *

In [2]:
data = get_date_list(data)
data = read_data(data)
data = run_engine(data)

if data["plot"]:
    plot_graph(data)
    
print('-----------------------------------------------------')
print_report(data)
print('-----------------------------------------------------')
print(data['report_df'].groupby(['date_val'])['pls'].sum())
print('-----------------------------------------------------')
print(data['report_df']['pls'].value_counts())
print('-----------------------------------------------------')
data['report_df']

 14%|████                          | 611950/4513175 [1:55:10<13:58:26, 77.55it/s]

pls    0.0061
dtype: float64
--------------------------------------
                  date     pls
0  2021-01-05 01:00:15  0.0028
1  2021-01-05 08:11:33 -0.0012
2  2021-01-05 14:15:38 -0.0002
3  2021-01-06 00:56:57  0.0031
4  2021-01-06 03:16:50 -0.0001
5  2021-01-06 04:57:27 -0.0012
6  2021-01-06 05:19:05 -0.0008
7  2021-01-06 17:14:30  0.0028
8  2021-01-07 06:41:02 -0.0004
9  2021-01-07 14:30:11 -0.0011
10 2021-01-07 16:58:25 -0.0006
11 2021-01-07 19:27:44 -0.0002
12 2021-01-08 09:59:11  0.0013
13 2021-01-08 15:57:21  0.0002
14 2021-01-08 16:09:24 -0.0006
15 2021-01-08 16:19:14 -0.0010
16 2021-01-08 16:24:25 -0.0011
17 2021-01-08 16:37:00 -0.0011
18 2021-01-11 08:09:21  0.0055


 14%|████                          | 613283/4513175 [1:55:27<12:14:09, 88.53it/s]

KeyboardInterrupt

